# Project 1: Linear and Ploynomial Multivariate Regression

This notebook estimates car MPG based on other data about the car. It receives its data from a CSV file (`auto-mpg.data`) and stores it in a Pandas DataFrame. Basic imputation is performed to remove the NaN values found in the horsepower column, and the data is standardized. Both a linear and polynomial multivariate regression algorithms are used to predict the MPG of the car.

In [1]:
import pandas as pd
import numpy as np

## Open the file into a Pandas DataFrame

In [2]:
def create_data_frame(fname):
    data = pd.read_table(fname, header=None, delim_whitespace=True,
                         names=["mpg", "cylinders", "displacement", "horsepower",
                                "weight", "acceleration", "model year", "origin", "car name"])
    return data

In [3]:
data = create_data_frame("auto-mpg.data")

## Contents of `auto-mpg.data`

Contents are listed as pairs of column names and the type of data in the column:

1. **mpg**:       continuous
2. __cylinders__:    multi-valued discrete
3. __displacement__:  continuous
4. __horsepower__:    continuous
5. __weight__:        continuous
6. __acceleration__:  continuous
7. __model year__:    multi-valued discrete
8. __origin__:        multi-valued discrete
9. __car name__:      string (unique for each instance)

There are 398 rows (instances), each with these 9 attributes. The horsepower column is also known to have 6 NaN values.

The following cell shows the first 10 rows of the data.

In [4]:
data.head(10)

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130.0,3504.0,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165.0,3693.0,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150.0,3436.0,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150.0,3433.0,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140.0,3449.0,10.5,70,1,ford torino
5,15.0,8,429.0,198.0,4341.0,10.0,70,1,ford galaxie 500
6,14.0,8,454.0,220.0,4354.0,9.0,70,1,chevrolet impala
7,14.0,8,440.0,215.0,4312.0,8.5,70,1,plymouth fury iii
8,14.0,8,455.0,225.0,4425.0,10.0,70,1,pontiac catalina
9,15.0,8,390.0,190.0,3850.0,8.5,70,1,amc ambassador dpl


## Imputation

To impute the NaN values in the _horsepower_ column, replacement with the average value is used so that the data is not removed.

In [5]:
def clean_Nan(data):
    num_cols = data.shape[1]
    num_rows = data.shape[0]
    for col in range(num_cols-1):
        elem_list = []
        col_sum = 0
        num_items = 0
        for row in range(num_rows):
            if type(data.iloc[row, col]) is int or type(data.iloc[row, col]) is float:
                if np.isnan(data.iloc[row, col]):
                    elem_list.append((row, col))
                else:
                    col_sum += data.iloc[row, col]
                    num_items += 1
            elif type(data.iloc[row, col]) is str:
                try:
                    fdata = float(data.iloc[row, col])
                except ValueError:
                    fdata = np.nan
                if np.isnan(fdata):
                    elem_list.append((row, col))
                else:
                    data.iloc[row, col] = fdata
                    col_sum += data.iloc[row, col]
                    num_items += 1
        if num_items > 0:
            avg = col_sum / num_items
            for r, c in elem_list:
                data.iloc[r, c] = avg

The `car name` column is dropped as it provides no useful information for the algorithm. Afterwards, a `const` column is added to simplify prediction later.

In [6]:
clean_Nan(data)
data = data.iloc[:, :-1]
data["const"] = 1
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,const
0,18.0,8,307.0,130,3504.0,12.0,70,1,1
1,15.0,8,350.0,165,3693.0,11.5,70,1,1
2,18.0,8,318.0,150,3436.0,11.0,70,1,1
3,16.0,8,304.0,150,3433.0,12.0,70,1,1
4,17.0,8,302.0,140,3449.0,10.5,70,1,1
5,15.0,8,429.0,198,4341.0,10.0,70,1,1
6,14.0,8,454.0,220,4354.0,9.0,70,1,1
7,14.0,8,440.0,215,4312.0,8.5,70,1,1
8,14.0,8,455.0,225,4425.0,10.0,70,1,1
9,15.0,8,390.0,190,3850.0,8.5,70,1,1


## Generate Statistics

Statistics are obtained for the imputated data to aid in determining a standardization process. The following statistics are calculated:
* Mean
* Standard Deviation
* Min/Max
* Quartiles
* Number of Entries

In [7]:
def get_stats(data):
    # Makes a 8x8 array of statistics
    # Note: car names are excluded from this
    stats = np.empty([8,8])
    df = data.values[:,:-1]
    inds = np.asarray(np.where(df == '?'))
    for r, c in inds.T:
        df[r, c] = np.nan
    df = df.astype(float)
    stats[:,0] = np.mean(df, axis=0)
    stats[:,1] = np.std(df, axis=0)
    stats[:,2] = df.min(axis=0)
    stats[:,3] = df.max(axis=0)
    stats[:,4] = np.percentile(df, 25, axis=0)
    stats[:,5] = np.percentile(df, 50, axis=0)
    stats[:,6] = np.percentile(df, 75, axis=0)
    stats[:,7].fill(df.shape[0])
    stats = pd.DataFrame(stats, index=data.columns[:-1], columns=["Mean", "Std", "Min", "Max", "25 Percentile", "50 Percentile", "75 Percentile", "Num Elems"])
    return stats

In [8]:
stats = get_stats(data)

In [9]:
print(pd.DataFrame(stats))

                     Mean         Std     Min     Max  25 Percentile  \
mpg             23.514573    7.806159     9.0    46.6         17.500   
cylinders        5.454774    1.698866     3.0     8.0          4.000   
displacement   193.425879  104.138764    68.0   455.0        104.250   
horsepower     104.469388   38.151168    46.0   230.0         76.000   
weight        2970.424623  845.777234  1613.0  5140.0       2223.750   
acceleration    15.568090    2.754222     8.0    24.8         13.825   
model year      76.010050    3.692978    70.0    82.0         73.000   
origin           1.572864    0.801047     1.0     3.0          1.000   

              50 Percentile  75 Percentile  Num Elems  
mpg                    23.0         29.000      398.0  
cylinders               4.0          8.000      398.0  
displacement          148.5        262.000      398.0  
horsepower             95.0        125.000      398.0  
weight               2803.5       3608.000      398.0  
acceleration   

## Standardization

For standardization, two techniques will be used:

1. For continuous features, the standardized output will be the z-score of the input.
2. For discrete features, the data will simply be subtracted from so that the minimum value becomes 1.

In [10]:
def standardize(data, stats):
    cont = ["displacement", "horsepower", "weight", "acceleration"]
    discr = ["cylinders", "model year", "origin"]
    for label in cont:
        data[label] = data[label].apply(lambda x: (x - stats.loc[label, "Mean"]) / stats.loc[label, "Std"])
    for label in discr:
        data[label] = data[label].apply(lambda x: x - stats.loc[label, "Min"] + 1)

In [11]:
standardize(data, stats)
data

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,const
0,18.0,6.0,1.090604,6.691961e-01,0.630870,-1.295498,1.0,1.0,1
1,15.0,6.0,1.503514,1.586599e+00,0.854333,-1.477038,1.0,1.0,1
2,18.0,6.0,1.196232,1.193426e+00,0.550470,-1.658577,1.0,1.0,1
3,16.0,6.0,1.061796,1.193426e+00,0.546923,-1.295498,1.0,1.0,1
4,17.0,6.0,1.042591,9.313113e-01,0.565841,-1.840117,1.0,1.0,1
5,15.0,6.0,2.262118,2.451579e+00,1.620492,-2.021656,1.0,1.0,1
6,14.0,6.0,2.502182,3.028233e+00,1.635863,-2.384735,1.0,1.0,1
7,14.0,6.0,2.367746,2.897175e+00,1.586204,-2.566274,1.0,1.0,1
8,14.0,6.0,2.511784,3.159290e+00,1.719809,-2.021656,1.0,1.0,1
9,15.0,6.0,1.887617,2.241887e+00,1.039961,-2.566274,1.0,1.0,1


## Split Data into Training and Testing Sets

The data is divided so that 75% of it is used for training, and the remaining 25% is used for testing.

The data is divided randomly to prevent bias.

In [12]:
num_rows = data.shape[0]
div = num_rows // 5
train_max = 4 * div
inds = np.random.choice(range(num_rows), size=train_max, replace=False)
test_inds = [i for i in range(num_rows) if i not in inds]
train = data.iloc[inds.tolist(), :]
test = data.iloc[test_inds, :]

## Split Data into Inputs and Outputs

The output data is separated from the input data, and all data is converted to `numpy` arrays of floats to simplify later calculations.

In [13]:
X_train = train.loc[:, "cylinders":"const"].values.astype(float)
r_train = train.loc[:, "mpg"].values.astype(float)
X_test = test.loc[:, "cylinders":"const"].values.astype(float)
r_test = test.loc[:, "mpg"].values.astype(float)

## Training for Linear Regression

A standard multivariate linear regression algorithm is used. The equation for the weights is as follows:
$$
w = (X^{T}X)^{-1}X^{T}r
$$

In [14]:
def linreg_train(X, r):
    return np.matmul(np.matmul(np.linalg.inv(np.matmul(X.T, X)), X.T), r)

Ensures the input contains a 1 at its end to simplify the prediction.

In [15]:
def linreg_predict(X, weights):
    if len(X) == len(weights):
        X_pred = X[:]
    elif len(X) == len(weights)-1:
        X_pred = np.append(X, 1)
    else:
        raise TypeError("weights (size {}) and X (size {}) have incompatible sizes.\nSizes should either be the same, or X should be one element smaller than weights.".format(len(weights), len(X)))
    return np.dot(weights, X_pred)

Calculates both the mean squared error and the percent error for the provided model given input `X` and expected output `r`.

In [16]:
def error_linreg(X, r, weights):
    scores = []
    for data, result in zip(X, r):
        y = linreg_predict(data, weights)
        scores.append((y-result)**2)
    scores = np.array(scores)
    lsquare_error = np.average(scores)
    scores = []
    for data, result in zip(X, r):
        y = linreg_predict(data, weights)
        curr_error = abs((y - result) / result) * 100
        scores.append(curr_error)
    scores = np.array(scores)
    avg_percent_error = np.average(scores)
    return lsquare_error, avg_percent_error

In [17]:
weights = linreg_train(X_train, r_train)
weights

array([-0.62788944,  2.30013598, -0.33163155, -5.66491298,  0.3941278 ,
        0.73743211,  1.57943072, 17.82706813])

## Check Training Error

In [18]:
lsquare, perror = error_linreg(X_train, r_train, weights)
print("Mean Squared Error on Training = {}".format(lsquare))
print("Percent Error on Training = {}".format(perror))

Mean Squared Error on Training = 10.998539856859493
Percent Error on Training = 11.591004705836975


## Testing for Linear Regression

Mean Squared Error will be used as the main testing algorithm.

A basic percent error will also be used.

In [19]:
lsquare_test, perror_test = error_linreg(X_test, r_test, weights)
print("Mean Squared Error on Training = {}".format(lsquare_test))
print("Percent Error on Training = {}".format(perror_test))

Mean Squared Error on Training = 10.94012308372772
Percent Error on Training = 11.208914932808282


## Training for Polynomial Regression

The multivariate polynomial regression algorithm is implemented by calculating all powers of each variable from 1 up to the degree of the polynomial (i.e. for a quadratic regression, it calculates square of each feature and preserves the original values). It adds the extra data into a new data array. Then, the linear regression algorithm from above is applied to the expanded dataset to get the polynomial regression's weights.

This function reads in the same type of data that was passed to the linear regression algorithm and expands it to work for the polynomial regression algorithm.

In [20]:
def _expand_data_to_degree(data, degree=2):
    try:
        num_cols = data.shape[1]-1
        final_data = np.empty((data.shape[0],0))
        for col in range(num_cols):
            for i in range(degree-1):
                new_col = np.power(data[:, col], degree-i)
                final_data = np.column_stack((final_data, new_col))
            final_data = np.column_stack((final_data, data[:, col]))
        final_data = np.column_stack((final_data, data[:, -1]))
    except IndexError:
        num_cols = len(data)-1
        final_data = np.empty((0,))
        for col in range(num_cols):
            for i in range(degree-1):
                new_col = data[col]**(degree-i)
                final_data = np.append(final_data, new_col)
            final_data = np.append(final_data, data[col])
        final_data = np.append(final_data, data[-1])
    return final_data

In [21]:
def polyreg_train(X, r, degree=2):
    X_poly = _expand_data_to_degree(X, degree)
    return (linreg_train(X_poly, r), degree)

In [22]:
def polyreg_predict(X, weights):
    X_poly = _expand_data_to_degree(X, weights[1])
    return linreg_predict(X_poly, weights[0])

Calculates both the mean squared error and the percent error for the provided model given input `X` and expected output `r`.

In [23]:
def error_polyreg(X, r, weights):
    scores = []
    for data, result in zip(X, r):
        y = polyreg_predict(data, weights)
        scores.append((y-result)**2)
    scores = np.array(scores)
    lsquare_error = np.average(scores)
    scores = []
    for data, result in zip(X, r):
        y = polyreg_predict(data, weights)
        curr_error = abs((y - result) / result) * 100
        scores.append(curr_error)
    scores = np.array(scores)
    avg_percent_error = np.average(scores)
    return lsquare_error, avg_percent_error

In [37]:
def cross_validate(X_train, r_train):
    errors = []
    for degree in range(1, 5):
        errors.append([])
    for i in range(10):
        num_tr = X_train.shape[0]
        div_tr = num_rows // 4
        tr_max = 3 * div
        tr_inds = np.random.choice(range(num_tr), size=tr_max, replace=False)
        tr_valid_inds = [i for i in range(num_tr) if i not in tr_inds]
        X_train_tr = X_train[tr_inds, :]
        X_valid_tr = X_train[tr_valid_inds, :]
        r_train_tr = r_train[tr_inds]
        r_valid_tr = r_train[tr_valid_inds]
        for degree in range(1, 5):
            poly_weights = polyreg_train(X_train_tr, r_train_tr, degree=degree)
            train_error, _ = error_polyreg(X_train_tr, r_train_tr, poly_weights)
            errors[degree-1].append(train_error)
            valid_error, _ = error_polyreg(X_valid_tr, r_valid_tr, poly_weights)
            errors[degree-1].append(valid_error)
    avg_errors = []
    for el in errors:
        el = np.array(el)
        avg_errors.append(np.average(el))
    avg_errors = np.array(avg_errors)
    best_degree = np.argmin(avg_errors)+1
    print("Cross Validation suggests the best polynomial degree is {}".format(best_degree))
    return best_degree

## Check Training Error

In [39]:
degree = cross_validate(X_train, r_train)
poly_weights = polyreg_train(X_train, r_train, degree=degree)
lsquare_poly, perror_poly = error_polyreg(X_train, r_train, poly_weights)
print("Mean Squared Error on Training = {}".format(lsquare_poly))
print("Percent Error on Training = {}".format(perror_poly))

Cross Validation suggests the best polynomial degree is 2
Mean Squared Error on Training = 7.499732745209701
Percent Error on Training = 8.736519702846165


## Testing for Linear Regression
Mean Squared Error will be used as the main testing algorithm.

A basic percent error will also be used.

In [40]:
lsquare_testpoly, perror_testpoly = error_polyreg(X_test, r_test, poly_weights)
print("Mean Squared Error on Training = {}".format(lsquare_testpoly))
print("Percent Error on Training = {}".format(perror_testpoly))

Mean Squared Error on Training = 7.568069679062766
Percent Error on Training = 8.012203276148279
